In [44]:
import pandas as pd
from sklearn.model_selection import TimeSeriesSplit, train_test_split
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [11]:
df=pd.read_csv("dataset.csv")

In [13]:
df.columns

Index(['Unnamed: 0', 'activity', 'appCat.builtin', 'appCat.communication',
       'appCat.entertainment', 'appCat.finance', 'appCat.game',
       'appCat.office', 'appCat.other', 'appCat.social', 'appCat.travel',
       'appCat.unknown', 'appCat.utilities', 'appCat.weather', 'call',
       'circumplex.arousal', 'circumplex.valence', 'mood', 'screen', 'sms',
       'id', 'day', 'target_mood'],
      dtype='object')

In [15]:
#sort by data 
df['day'] = pd.to_datetime(df['day'])
df = df.sort_values(by='day')

In [19]:
df['day']

541   2014-03-13
268   2014-03-13
396   2014-03-13
127   2014-03-14
489   2014-03-14
         ...    
809   2014-06-03
810   2014-06-04
811   2014-06-05
812   2014-06-06
813   2014-06-07
Name: day, Length: 1169, dtype: datetime64[ns]

In [17]:
len(df['day'].unique())

86

In [6]:
df = df.drop(columns=['Unnamed: 0','id','day'])

In [7]:
#seperate 80/20
train_df, temp_df = train_test_split(df, test_size=0.2, shuffle=False)

#seperate 20 into 50/50
val_df, test_df = train_test_split(temp_df, test_size=0.5, shuffle=False)

print("Train shape:", train_df.shape)
print("Validation shape:", val_df.shape)
print("Test shape:", test_df.shape)

Train shape: (935, 20)
Validation shape: (117, 20)
Test shape: (117, 20)


In [21]:
X_train = train_df.drop(['target_mood'], axis=1).values
y_train = train_df['target_mood'].values

X_val = val_df.drop(['target_mood'], axis=1).values
y_val = val_df['target_mood'].values

X_test = test_df.drop(['target_mood'], axis=1).values
y_test = test_df['target_mood'].values

In [9]:
#validate that time series is correct --> I checked this but now this dindt work because days is remvmove for the model 
"""print(train_df['day'].min())
print(val_df['day'].min())
print(test_df['day'].min())"""

"print(train_df['day'].min())\nprint(val_df['day'].min())\nprint(test_df['day'].min())"

In [31]:
#time steps --> batch size is hyperparameter 
time_steps = 86
batch_size = 16

# use predifined function 
train_dataset = tf.keras.preprocessing.timeseries_dataset_from_array(
    data=X_train,
    targets=y_train,
    sequence_length=time_steps,
    sampling_rate=1,    # sample rate
    batch_size=batch_size,
    shuffle=True        #  optional 
)

In [25]:

val_dataset = tf.keras.preprocessing.timeseries_dataset_from_array(
    data=X_val,
    targets=y_val,
    sequence_length=time_steps,
    sampling_rate=1,
    batch_size=batch_size,
    shuffle=False       
)


In [27]:
test_dataset = tf.keras.preprocessing.timeseries_dataset_from_array(
    data=X_test,
    targets=y_test,
    sequence_length=time_steps,
    sampling_rate=1,
    batch_size=batch_size,
    shuffle=False
)


In [62]:
# batch_size, time_steps, num_features

for batch_data, batch_targets in train_dataset.take(1):
    print("Batch data shape:", batch_data.shape)  

Batch data shape: (16, 86, 19)


In [39]:
# define model
model = Sequential()

#shape is based on number of unique timestamps (86) and number of features (19)
model.add(LSTM(units=50, input_shape=(86, 19)))
model.add(Dropout(0.2))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()

C:\Users\youpz\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                   │ (None, 50)             │        14,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,051 (54.89 KB)

 Trainable params: 14,051 (54.89 KB)

 Non-trainable params: 0 (0.00 B)

In [64]:
#train the model
history = model.fit(train_dataset, epochs=20, validation_data=val_dataset)

Epoch 1/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 1.3311 - val_loss: 2.9800
Epoch 2/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - loss: 1.2861 - val_loss: 2.8723
Epoch 3/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 1.4509 - val_loss: 3.4309
Epoch 4/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 1.3176 - val_loss: 3.1159
Epoch 5/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 1.3247 - val_loss: 3.1794
Epoch 6/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 1.2259 - val_loss: 3.4009
Epoch 7/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 1.1308 - val_loss: 3.2314
Epoch 8/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 1.2683 - val_loss: 3.4157
Epoch 9/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 1.2211 - val_loss: 2.6251
Epoch 10/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 1.2516 - val_loss: 2.6019
Epoch 11/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 1.3428 - val_loss: 2.6411
Epoch 12/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 1.1

In [52]:
#evaluating of the code
y_pred = model.predict(test_dataset)
y_pred = y_pred.flatten()
y_true = []

#make the frame fit the tensor format
for _, y in test_dataset:
    y_true.extend(y.numpy())

#make true value
y_true = np.array(y_true)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step


In [56]:
#calculat the scores
mse = mean_squared_error(y_true, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_true, y_pred)

print(f"Test MSE:  {mse:.4f}")
print(f"Test RMSE: {rmse:.4f}")
print(f"Test MAE:  {mae:.4f}")

Test MSE:  0.9622
Test RMSE: 0.9809
Test MAE:  0.8606
